# input data 전처리 함수

In [20]:
import re

def tagging_pos_input_data(komoran, input_str):

    input_token_komoran = komoran.get_plain_text(input_str).split(' ')
    # print('코모란:', input_token_komoran)

    df_komoran = pd.DataFrame([x.split('/') for x in input_token_komoran], columns=['형태소', '주석'])

    # df_komoran
    
    # 정규식 패턴
    pattern_english = r'^[a-zA-Z]+$'
    pattern_number = r'\d+'
    pattern_symbol = r'[!@#$%^&*()_+-={}\[\]:;"\'<>,.?/~`|]'
    pattern_empty = r'\s+'

    # 제거할 형태소 리스트
    list_english = []
    list_number = []
    list_symbol = []
    list_empty = []

    ## 영어 제거
    for index, row in df_komoran.iterrows():
        if re.match(pattern_english, row['형태소']):
            list_english.append(index)

    df_remove_english = df_komoran.drop(list_english)


    ## 숫자 제거
    for index, row in df_remove_english.iterrows():
        if re.match(pattern_number, row['형태소']):
            list_number.append(index)

    df_remove_number = df_remove_english.drop(list_number)


    ## 기호 제거
    for index, row in df_remove_number.iterrows():
        if re.match(pattern_symbol, row['형태소']):
            list_symbol.append(index)

    df_remove_symbol = df_remove_number.drop(list_symbol)


    ## 공백 제거
    for index, row in df_remove_symbol.iterrows():
        if re.match(pattern_empty, row['형태소']):
            list_empty.append(index)

    df_data = df_remove_symbol.drop(list_empty)

    # df_data['형태소'].unique()

    ## 미사용 형태 주석 제거
    using_pos = ['NP', 'NNG', 'XSA', 'EP', 'EF', 'JX', 'NNP', 'JKB', 'XSV', 'EC', 'VX', 'ETM', 'XSN', 'VCP', 'VV', 'VA', 'JKS', 'MAG', 'XR', 'JC', 'NA', 'VCN', 'MM', 'ETN', 'MAJ', 'JKG', 'JKC', 'IC', 'NNB', 'NR', 'JKO', 'XPN', 'JKQ']

    df_data = df_data[df_data['주석'].isin(using_pos)]

    # print(df_data['주석'].unique())
    
    ## stopwords 리스트 생성
    txt_tmp = open("./koreanStopwords.txt", 'r', encoding="utf-8")

    stopwords = []
    while True:
        line = txt_tmp.readline()
        if not line: break
        form, pos, _ = line.split()
        stopwords.append([form, pos])

    txt_tmp.close()

    ## stopwords 제거
    for f, p in stopwords:
        df_data = df_data[~((df_data["형태소"] == f) & (df_data["주석"] == p))]

    return df_data

In [3]:
# ## test 
# result = tagging_pos_input_data("음 아안녕하세요. 전 저는 <name1>입니다. 저는 학생입니다. <name2>대학교 학교에 국 한국어를 배워요. 저는 지금? 좋아요. 어 제가쪽이 음 네 명이에요. 아버지하고 어머니하고 누나하고 저이에요. 아버지는 선생님이에요. 음 아버지는 지금? 좋아요. 어 어머니 어머니는 요리사이에요 예요(이에요). 엄 으음 뜨 뜨라 뜨라 드 드라마를 좋아요. 어 누나 누나 누나가 회사원이에요. 저 저 운도를? 해요. 아주 좋아해요.")

# result

TypeError: tagging_pos_input_data() missing 1 required positional argument: 'input_str'

# 단어 사전 기반 점수 

### 사전 데이터 전처리

In [4]:
#판다스로 어휘 문법 등급 목록 가져오기
import pandas as pd
df = pd.read_excel("./2017년 국제 통용 한국어 표준 교육과정 적용 연구(4단계) 어휘, 문법 등급 목록.xlsx", sheet_name = [0,1])

#어휘와 문법의 구성 요소만 확인할 수 있는 데이터프레임 생성
df_voca = pd.concat([df[0]['어휘'], df[1]['대표형']])

#확인
df_voca

0          가게
1        가격02
2        가구02
3         가깝다
4        가다01
        ...  
331      이라고는
332    -는다던가1
333     -으래서야
334     -으리라고
335      -자면2
Length: 10971, dtype: object

In [5]:
#값 내 숫자 제거
df_voca = df_voca.replace(r'\d+', '', regex=True)

#숫자 제거 확인
df_voca

0         가게
1         가격
2         가구
3        가깝다
4         가다
       ...  
331     이라고는
332    -는다던가
333    -으래서야
334    -으리라고
335      -자면
Length: 10971, dtype: object

In [6]:
#코모란 설치
!pip install PyKomoran

!python3 --version

Python 3.11.4


In [7]:
#코모란 라이브러리 가져오기 및 확인
from PyKomoran import *
komoran = Komoran("EXP")
print(komoran.get_plain_text("KOMORAN은 한국어 형태소 분석기입니다."))

KOMORAN/SL 은/JX 한국어/NNP 형태소/NNP 분석기/NNG 이/VCP ㅂ니다/EF ./SF


In [8]:
#한국어 불용어 사전 가져오기
stop = pd.read_table("./koreanStopwords.txt", names =["단어", "형태소", "수치"])
print(stop)

    단어  형태소        수치
0    이  VCP  0.018280
1    있   VA  0.011699
2    하   VV  0.009774
3    것  NNB  0.009733
4    들  XSN  0.006898
..  ..  ...       ...
95   원  NNB  0.000492
96   잘  MAG  0.000491
97  통하   VV  0.000487
98  소리  NNG  0.000486
99   놓   VX  0.000483

[100 rows x 3 columns]


In [9]:
#불용어 사전에서 필요한 정보만 추출하기
stop['단어']

0      이
1      있
2      하
3      것
4      들
      ..
95     원
96     잘
97    통하
98    소리
99     놓
Name: 단어, Length: 100, dtype: object

In [83]:
#어휘 문법 사전의 구성 요소들을 형태소 형태로 변경 + 불용어 제거
stopwords = [[stop['단어'], '-']]

voca_list = []

for w in df_voca:
  newdf = komoran.get_morphes_by_tags(w, tag_list=['NNG', 'IC', 'EF', 'JKB', 'NNP'])

  for w in newdf:
    if w not in stopwords:
      voca_list.append(w)

#형태소 형태로 전환된 새로운 어휘 문법 데이터프레임 생성
vocadf = pd.Series(voca_list)

#확인
print('상위 20개:\n',vocadf.head(20))
print('상위 20개:\n',vocadf.tail(20))

#데이터 프레임 저장
vocadf.to_csv('./vocadf.csv', index=False)

상위 20개:
 0      가격
1      가구
2      가방
3      가수
4      가요
5     가운데
6      가족
7      갈비
8     갈비탕
9       감
10     감기
11     감사
12      값
13     거실
14     건강
15    건너편
16     건물
17     게임
18     겨울
19     계속
dtype: object
상위 20개:
 7814     테
7815     테
7816    에도
7817     을
7818     랑
7819    건대
7820     디
7821    자니
7822    구려
7823     오
7824    구려
7825     네
7826    구만
7827     소
7828    일쑤
7829     짝
7830    마당
7831     에
7832     한
7833     에
dtype: object


### 사전 기반 발음 점수 Generator 모듈 (50점 만점의 점수 리턴)

In [104]:
# # vocafd에서 word를 찾아서 있으면 1, 없으면 0 리턴함
# def get_pronunciation_score_for_word_by_vocadf(word):
#     if voca_list.count(word):
#         print("찾음!")
#         return 1
#     else :
#         print("못찾음!")
#         return 0

# 사전 기반 발음 점수 -> 50점 만점
def get_pronunciation_score_by_vocadf(wordList):
    matchedCount = 0
    for word in wordList:
        if voca_list.count(word) != 0:
            matchedCount = matchedCount + 1
            # print(matchedCount)

    # print(1-(matchedCount/len(wordList))*100)
    return (matchedCount/len(wordList))*50



# print("당신의 사전 기반 발음 점수는: ")
# print(score)

# print("당신의 사전 기반 발음 점수는: ")
# print(score2)


['가격', '가구', '가방', '가수', '가요', '가운데', '가족', '갈비', '갈비탕', '감', '감기', '감사', '값', '거실', '건강', '건너편', '건물', '게임', '겨울', '계속', '계절', '계획', '고기', '고양이', '고향', '곳', '공부', '공원', '공책', '공항', '과일', '교실', '교통', '구', '구경', '구두', '구월', '국적', '귀', '귤', '그래', '그릇', '그림', '그저께', '그저께', '극장', '근처', '글', '글쎄요', '금요일', '기간', '기분', '기숙사', '기차', '길', '김밥', '김치', '김치찌개', '꽃', '나', '나라', '나무', '나중', '날', '날', '날씨', '날짜', '남대문', '남동생', '남산', '남자', '남편', '낮', '내년', '내일', '내일', '냉면', '노래', '노래방', '농구', '누나', '눈', '눈', '뉴스', '다', '다리', '다음', '단어', '달', '달', '담배', '대답', '대사관', '대학', '대학교', '대학생', '대화', '도서관', '도착', '독일', '돈', '동대문', '동생', '동안', '된장찌개', '뒤', '드라마', '등산', '딸', '딸기', '때', '떡볶이', '라면', '러시아', '마음', '만', '말', '말레이시아', '말씀', '맛', '매일', '머리', '메뉴', '며칠', '모레', '모레', '모자', '목', '목요일', '목욕탕', '몸', '다', '못', '몽골', '문', '문화', '물', '물건', '미국', '미안', '미용실', '밑', '바나나', '바다', '바람', '바지', '박물관', '밖', '반', '반', '발', '밤', '밥', '방', '방학', '배', '배', '배', '배우', '백', '백화점', '버스', '번호', '베트남', '병', '병', '병원', '보통', '보

# 오타 기반 발음 점수 

In [97]:
! pip3 install symspellpy-ko


[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: /usr/local/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


In [98]:
from symspellpy_ko import KoSymSpell, split_syllables, join_jamos, Verbosity

def sym_spell():
    sp = KoSymSpell()
    sp.load_korean_dictionary(decompose_korean=True, load_bigrams=True)
    return sp

def test_lookup(sym_spell, term):
    lookup = sym_spell.lookup(term, Verbosity.CLOSEST)[0]
    assert lookup.term == "안녕하세요"
    assert lookup.distance == 1

In [99]:
sp = sym_spell()

lookup = sp.lookup("안뇽하십니까", Verbosity.CLOSEST)[0].term

print(lookup)

안녕하십니까


### 오타 기반 발음 점수 Generator 모듈 (50점 만점의 점수 리턴)

In [100]:
# word를 받아서 word의 오타를 제거하고, 제거된 char의 비율을 계산함. 
def get_pronunciation_score_by_correction_for_word(word):
    try :
        fixed_text = sp.lookup(word, Verbosity.CLOSEST)[0].term
        # count how many characters are different
        count = 0
        for i in range(len(word)):
            if word[i] not in fixed_text:
                count += 1
        score = (1 - count/len(word)) 
        # print(fixed_text)
    except :
        # print("Cannot find a proper text")
        return 0

    return score
    

# 리스트를 받아서 리스트 내 아이템 하나씩 스코어 함수 호출. (50점 만점)
def get_pronunciation_score_by_correction(list):
    score = 0
    for word in list:
        score += get_pronunciation_score_by_correction_for_word(word)
        
    return score/len(list)*50


get_pronunciation_score_by_correction(["아야", "맛난거"])

33.333333333333336

두 점수 종합하는 function

In [101]:
# Komoran 준비 ( Java 에러 종종 나서 분리)
komoran = Komoran("EXP")


In [124]:
def get_pronunciation_score(text):
    list_pos_tagged = tagging_pos_input_data(komoran, text)['형태소']
    # print(list_pos_tagged)
    tokenized_list = text.split(' ')
    # print(tokenized_list)
    score_by_voca = get_pronunciation_score_by_vocadf(list_pos_tagged)
    score_by_correction = get_pronunciation_score_by_correction(tokenized_list)


    print(f"단어 사전 기반 점수: {score_by_voca}")
    print(f"오타 교정 기반 점수: {score_by_correction}")
    print(f"종합 발음 점수: {score_by_voca+score_by_correction}")

    return score_by_correction


# 발음 좋은 케이스
print("발음좋음 : 안녕하세요, 제 이름은 <name1>입니다. 저는 수학 수업이 정말 좋아요. 저는 앞으로도 학습하며 발전해 나아가고 싶습니다. 저는 열심히 공부해서 사람들을 돕고 살고 싶어요.")
get_pronunciation_score ("안녕하세요, 제 이름은 <name1>입니다. 저는 수학 수업이 정말 좋아요. 저는 앞으로도 학습하며 발전해 나아가고 싶습니다. 저는 열심히 공부해서 사람들을 돕고 살고 싶어요.")
print("발음보통 : 안녕하세요 어 저는 베트남 온 유학생 <name>라고 합니다 어 제 전공은 신소재 공학이고 아직 일 학년 학생입니다 어 제 교향은 베트남 호치민 도시입니다 어 호치민 도시는 어 아주 아름다운 도시이고 호치민에 사람이 많이 있습니다 그리고 베트남 베트남은 제 생각에는 아 가장 아름다운 국가입니다 저는 어릴 때부터 벌써 제 성격은 어 내적인 성격인데 지금은 조금 변했습니다 어 저는 지금 외 외성적인 성격이 가지고 있습니다 그래서 저는 항상 밖에 놀고 근데 저는 가장 친한 친구만 같이 놀습니다 어 그리고 음 제 어릴 때 꿈은 어 공학 공부할 수 있는 꿈입니다.")
get_pronunciation_score ("안녕하세요 어 저는 베트남 온 유학생 <name>라고 합니다 어 제 전공은 신소재 공학이고 아직 일 학년 학생입니다 어 제 교향은 베트남 호치민 도시입니다 어 호치민 도시는 어 아주 아름다운 도시이고 호치민에 사람이 많이 있습니다 그리고 베트남 베트남은 제 생각에는 아 가장 아름다운 국가입니다 저는 어릴 때부터 벌써 제 성격은 어 내적인 성격인데 지금은 조금 변했습니다 어 저는 지금 외 외성적인 성격이 가지고 있습니다 그래서 저는 항상 밖에 놀고 근데 저는 가장 친한 친구만 같이 놀습니다 어 그리고 음 제 어릴 때 꿈은 어 공학 공부할 수 있는 꿈입니다")
print("발음보통 : <name1>입니다. <name2>에서왔습니다. 저는 두달 전에 한구에 왔습니다. 지금은 학교 근처 하숙집에 삽니다. 한국에는 X사람이 아주 많습니다. 하고 가게하고 커피숍도 많습니다. 그래서 여기는 아주 복잡합니다. 한국 사람이 아주 친절하고 재미있습니다. 여름에 여기는 날씨가 아주 덥지만 거기가 좋습니다. 음 한국에 야경도 아주 아름답습니다. 한국이 음식이 다양합니다. 음식이 맵지만 맛 아주 맛있습니다. 음 한 제 한국 생활을 아주 좋아합니다. 그리고 그래서 날마다 제 기분이 아주 좋습니다. 한국의 학교에 가고 싶습니다. 그래서 지금은 한국말을 열심히 공부합니다. 감사합니다.")
get_pronunciation_score ("<name1>입니다. <name2>에서왔습니다. 저는 두달 전에 한구에 왔습니다. 지금은 학교 근처 하숙집에 삽니다. 한국에는 X사람이 아주 많습니다. 하고 가게하고 커피숍도 많습니다. 그래서 여기는 아주 복잡합니다. 한국 사람이 아주 친절하고 재미있습니다. 여름에 여기는 날씨가 아주 덥지만 거기가 좋습니다. 음 한국에 야경도 아주 아름답습니다. 한국이 음식이 다양합니다. 음식이 맵지만 맛 아주 맛있습니다. 음 한 제 한국 생활을 아주 좋아합니다. 그리고 그래서 날마다 제 기분이 아주 좋습니다. 한국의 학교에 가고 싶습니다. 그래서 지금은 한국말을 열심히 공부합니다. 감사합니다.")
print("발음안좋음 : 쩌는 일하지 않아서 주푸에요 매일 지빤일가 난 다음에 어린이집에 드리고 가요 어린이집에 드리고 가자마자 한국 교실레 가서 한구꺼를 공부라고 가요 집베 있을 때 뗄레비션을 보믄셔 텔레비션을 뵤 보면서 바쁠 먹어요 집베 있을 때 항 번 한국 드라마를 짜주 봐요 그리고 아기를 아기가 집에 있을 때 어린이 프로그램을 보고 씨포요 ")
get_pronunciation_score ("안녕하세요 저는 학생입니다 대학교에서 한국어를 배워요")
print("발음안좋음 : 저는 한국에서 항국말을 배웁니다 뿌 뿌관야 허 여기는  ")
get_pronunciation_score ("저는 한국에서 항국말을 배웁니다 뿌 뿌관야 허 여기는 ")




발음좋음 : 안녕하세요, 제 이름은 <name1>입니다. 저는 수학 수업이 정말 좋아요. 저는 앞으로도 학습하며 발전해 나아가고 싶습니다. 저는 열심히 공부해서 사람들을 돕고 살고 싶어요.
단어 사전 기반 점수: 27.77777777777778
오타 교정 기반 점수: 42.159090909090914
종합 발음 점수: 69.93686868686869
발음보통 : 안녕하세요 어 저는 베트남 온 유학생 <name>라고 합니다 어 제 전공은 신소재 공학이고 아직 일 학년 학생입니다 어 제 교향은 베트남 호치민 도시입니다 어 호치민 도시는 어 아주 아름다운 도시이고 호치민에 사람이 많이 있습니다 그리고 베트남 베트남은 제 생각에는 아 가장 아름다운 국가입니다 저는 어릴 때부터 벌써 제 성격은 어 내적인 성격인데 지금은 조금 변했습니다 어 저는 지금 외 외성적인 성격이 가지고 있습니다 그래서 저는 항상 밖에 놀고 근데 저는 가장 친한 친구만 같이 놀습니다 어 그리고 음 제 어릴 때 꿈은 어 공학 공부할 수 있는 꿈입니다.
단어 사전 기반 점수: 21.721311475409834
오타 교정 기반 점수: 43.428030303030305
종합 발음 점수: 65.14934177844015
발음보통 : <name1>입니다. <name2>에서왔습니다. 저는 두달 전에 한구에 왔습니다. 지금은 학교 근처 하숙집에 삽니다. 한국에는 X사람이 아주 많습니다. 하고 가게하고 커피숍도 많습니다. 그래서 여기는 아주 복잡합니다. 한국 사람이 아주 친절하고 재미있습니다. 여름에 여기는 날씨가 아주 덥지만 거기가 좋습니다. 음 한국에 야경도 아주 아름답습니다. 한국이 음식이 다양합니다. 음식이 맵지만 맛 아주 맛있습니다. 음 한 제 한국 생활을 아주 좋아합니다. 그리고 그래서 날마다 제 기분이 아주 좋습니다. 한국의 학교에 가고 싶습니다. 그래서 지금은 한국말을 열심히 공부합니다. 감사합니다.
단어 사전 기반 점수: 23.451327433628318
오타 교정 기반 점수: 38.8062622

26.851851851851855